In [1]:
!rm *.xlsx
!rm *.xls
from google.colab import files
upload=files.upload()

rm: cannot remove '*.xlsx': No such file or directory
rm: cannot remove '*.xls': No such file or directory


Saving 04월 서울시 교통량 조사자료.xlsx to 04월 서울시 교통량 조사자료.xlsx
Saving 04월 서울시 교통량 조사자료-수집지점 좌표.xlsx to 04월 서울시 교통량 조사자료-수집지점 좌표.xlsx
Saving 전기차 충전소 설치현황_20210522.xls to 전기차 충전소 설치현황_20210522.xls


In [2]:
import requests
def getLatLng(address):
 
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    rest_api_key = 'b1e115113f1c6c2fa72a2d1fe70f70b6'
    header = {'Authorization': 'KakaoAK ' + rest_api_key}
 
    r = requests.get(url, headers=header)
 
    if r.status_code == 200:
        result_address = r.json()["documents"][0]["address"]
        
        result = result_address["y"], result_address["x"]
    else:
        result = "ERROR[" + str(r.status_code) + "]"
    
    return result

address = "서울특별시 강남구 자곡동 자곡로3길 21"
    
address_latlng = getLatLng(address)
print(address_latlng)

 


('37.4717696093503', '127.088116003682')


In [46]:
import numpy as np
import pandas as pd
traffic=pd.read_excel('04월 서울시 교통량 조사자료.xlsx')
traffic_latlon=pd.read_excel('04월 서울시 교통량 조사자료-수집지점 좌표.xlsx')
charger=pd.read_excel('전기차 충전소 설치현황_20210522.xls')

In [47]:
traffic_latlon.drop(['검지기 유형', '주소', '도로명 주소', '유입 방향'], axis=1, inplace=True)
charger.drop(['시구', '급속충전기(대)', '완속충전기(대)', '지원차종'], axis=1, inplace=True)

In [48]:
traffic=traffic.fillna(0)
traffic['교통량합']=0
traffic['not_null_num']=0

In [49]:
times=[]
for i in range(0, 24):
  times.append(str(i) + '시')

for i in range(traffic.shape[0]):
  for time in times:
    if (traffic.loc[i, time]!=0):
      traffic.loc[i, 'not_null_num']+=1
    traffic.loc[i, '교통량합']=traffic.loc[i, '교통량합'] + traffic.loc[i, time]
    
for time in times:
  traffic.drop([time], axis=1, inplace=True)

In [50]:
alphabets=['A', 'B', 'C', 'D', 'F']
nums=[24, 37, 21, 45, 9]
locations=[]
for i in range(5):
  for j in range(1, nums[i]+1):
    if j<10:
      location_str='0' + str(j)
    else:
      location_str=str(j)
    location_str=alphabets[i] + '-' + location_str
    if (location_str != 'C-02'): 
        locations.append(location_str)
    

for i in range(len(locations)):
  traffic_sum=0
  not_null_num_sum=0
  for j in range(0, 60):
    traffic_sum+=traffic[traffic['지점번호']==locations[i]].loc[60*i + j, '교통량합']
    not_null_num_sum+=traffic[traffic['지점번호']==locations[i]].loc[60*i + j, 'not_null_num']
  traffic.at[60*i, '교통량합']=traffic_sum
  traffic.at[60*i, 'not_null_num']=not_null_num_sum


In [51]:
for i in range(134, -1, -1):
  for j in range(1, 60):
    traffic.drop([i * 60 + j], axis=0, inplace=True)
traffic=traffic.reset_index()

In [52]:
traffic['위도']=traffic_latlon['위도']
traffic['경도']=traffic_latlon['경도']


In [53]:
drop_row_index=[]
for i in range(traffic.shape[0]):
  if traffic.loc[i, 'not_null_num']==0:
    drop_row_index.append(i)

traffic.drop(drop_row_index, axis=0, inplace=True)
traffic=traffic.reset_index()

traffic['교통량합']=traffic['교통량합']/traffic['not_null_num']
traffic.drop(['level_0','index', '일자', '요일', '방향', '구분', 'not_null_num'], axis=1, inplace=True)

In [54]:
traffic['위도diff']=traffic['위도'].round(4)
traffic['경도diff']=traffic['경도'].round(4)



,지점명,지점번호,교통량합,위도,경도,위도diff,경도diff
0,성산로(금화터널),A-01,1229.838889,37.568588,126.948436,37.5686,126.9484
1,사직로(사직터널),A-02,1352.365278,37.572298,126.962853,37.5723,126.9629
2,자하문로(자하문터널),A-03,570.393750,37.588831,126.968548,37.5888,126.9685
3,대사관로(삼청터널),A-04,238.295139,37.596359,126.984209,37.5964,126.9842
4,율곡로(안국역),A-05,1059.229861,37.576000,126.984342,37.5760,126.9843
...,...,...,...,...,...,...,...
126,동부간선도로,F-05,2830.484601,37.568685,127.076023,37.5687,127.0760
127,경부고속도로,F-06,4021.402475,37.493135,127.022533,37.4931,127.0225
128,분당수서로,F-07,2795.689341,37.497648,127.087195,37.4976,127.0872
129,강남순환로,F-08,1850.950000,37.449096,126.926169,37.4491,126.9262


In [56]:
east=127.2693
west=126.7341
north=37.7151
south=37.4133
traffic['위도diff']-=south
traffic['경도diff']-=west

traffic['위도diff']*=10000
traffic['경도diff']*=10000

traffic['위도diff']=traffic['위도diff'].astype(int)
traffic['경도diff']=traffic['경도diff'].astype(int)

In [57]:
traffic.to_csv("traffic.csv")

In [58]:
traffic

,지점명,지점번호,교통량합,위도,경도,위도diff,경도diff
0,성산로(금화터널),A-01,1229.838889,37.568588,126.948436,1553,2143
1,사직로(사직터널),A-02,1352.365278,37.572298,126.962853,1589,2288
2,자하문로(자하문터널),A-03,570.393750,37.588831,126.968548,1754,2344
3,대사관로(삼청터널),A-04,238.295139,37.596359,126.984209,1831,2501
4,율곡로(안국역),A-05,1059.229861,37.576000,126.984342,1627,2502
...,...,...,...,...,...,...,...
126,동부간선도로,F-05,2830.484601,37.568685,127.076023,1554,3418
127,경부고속도로,F-06,4021.402475,37.493135,127.022533,797,2883
128,분당수서로,F-07,2795.689341,37.497648,127.087195,842,3530
129,강남순환로,F-08,1850.950000,37.449096,126.926169,358,1920


In [76]:
def GaussianFilter(filter_size=5, sigma=3):
    kernel1d=np.zeros(filter_size, dtype=np.float32)
    for i in range(filter_size):
        kernel1d[i]=np.exp( -(((i-((filter_size-1) / 2))**2) / (2* (sigma**2))))
    kernel2d = np.outer(kernel1d, kernel1d.transpose())
    return kernel2d

map_array=np.zeros([ (int((north-south)*10000)), (int((east-west)*10000)) ])
#123개
filtersize=2501
middle=int((filtersize-1)/2)
sigma=50
gaussian=GaussianFilter(filtersize, sigma)

for i in range(traffic.shape[0]):
  row=int(traffic.loc[i, '위도'])
  col=int(traffic.loc[i, '경도'])
  traffic_point=traffic.loc[i, '교통량합']

  if (row-middle>0 and row+middle+1<map_array.shape[0]):
    if (col-middle>0 and col+middle+1<map_array.shape[1]):
      for i in range(filtersize):
        for j in range(filtersize):
          map_array[row-middle+i, col-middle+j]+=traffic_point * gaussian[i, j]


In [77]:
from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

lat, lon = 37.5335, 126.9896
zoom_start = 5



data=np.array([traffic['위도'], traffic['경도'], traffic['교통량합']])
data=np.transpose(data)
data=data.tolist()
m = folium.Map([lat, lon], tiles='stamentoner', zoom_start=10)

HeatMap(data).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [44]:
data = (
    np.random.normal(size=(100, 3)) *
    np.array([[1, 1, 1]]) +
    np.array([[48, 5, 1]])
)
data


array([[ 4.78525232e+01,  3.29486207e+00,  5.02105774e-01],
       [ 4.97230132e+01,  4.98110522e+00, -3.74532193e-03],
       [ 4.81397559e+01,  4.89648064e+00,  4.69111729e-02],
       [ 4.71039644e+01,  5.26021041e+00,  2.56975670e+00],
       [ 4.83823229e+01,  6.31014086e+00,  7.24931308e-01],
       [ 4.85805608e+01,  4.36129676e+00, -1.59538472e+00],
       [ 4.84085667e+01,  4.11744598e+00,  7.79860638e-01],
       [ 4.88871306e+01,  5.43873543e+00,  7.08538832e-01],
       [ 4.75737056e+01,  6.20848815e+00,  1.91610882e+00],
       [ 4.67482085e+01,  4.35018971e+00,  1.41367076e-02],
       [ 4.75395635e+01,  6.18952172e+00,  1.94235174e+00],
       [ 4.76316060e+01,  5.60435151e+00,  1.01748361e+00],
       [ 4.76201908e+01,  2.21739839e+00,  1.97054178e+00],
       [ 4.89556385e+01,  6.12001427e+00,  2.38658478e+00],
       [ 4.78829698e+01,  4.34965743e+00,  1.96175839e+00],
       [ 4.73169698e+01,  4.22786948e+00,  3.65051970e+00],
       [ 4.72053748e+01,  4.68035118e+00

In [70]:
data=np.array([traffic['위도'], traffic['경도'], traffic['교통량합']])
data=np.transpose(data)
data=data.tolist()

numpy.float64